In [20]:
import pandas as pd
import geopandas as gpd
import os
from sklearn.model_selection import train_test_split

import glob 

import numpy as np

import rasterio 

from rasterstats import zonal_stats, point_query

import matplotlib.pyplot as plt

import multiprocessing

In [1]:
%%capture
import sys

# If you're on Colab:
if 'google.colab' in sys.modules:
    DATA_PATH = 'https://raw.githubusercontent.com/LambdaSchool/DS-Unit-2-Kaggle-Challenge/master/data/'
    !pip install category_encoders==2.*

# If you're working locally:
else:
    DATA_PATH = 'https://raw.githubusercontent.com/LambdaSchool/DS-Unit-2-Kaggle-Challenge/master/data/'

In [3]:
def wrangle(X):
    """Wrangle train, validate, and test sets in the same way"""
    
    # Prevent SettingWithCopyWarning
    X = X.copy()
    
    # About 3% of the time, latitude has small values near zero,
    # outside Tanzania, so we'll treat these values like zero.
    X['latitude'] = X['latitude'].replace(-2e-08, 0)
    
    # When columns have zeros and shouldn't, they are like null values.
    # So we will replace the zeros with nulls, and impute missing values later.
    # Also create a "missing indicator" column, because the fact that
    # values are missing may be a predictive signal.
    cols_with_zeros = ['longitude', 'latitude', 'construction_year', 
                       'gps_height', 'population']
    for col in cols_with_zeros:
        X[col] = X[col].replace(0, np.nan)
        X[col+'_MISSING'] = X[col].isnull()
            
    # Drop duplicate columns
    duplicates = ['quantity_group', 'payment_type']
    X = X.drop(columns=duplicates)
    
    # Drop recorded_by (never varies) and id (always varies, random)
    unusable_variance = ['recorded_by']
    X = X.drop(columns=unusable_variance)
    
    # Convert date_recorded to datetime
    X['date_recorded'] = pd.to_datetime(X['date_recorded'], infer_datetime_format=True)
    
    # Extract components from date_recorded, then drop the original column
    X['year_recorded'] = X['date_recorded'].dt.year
    X['month_recorded'] = X['date_recorded'].dt.month
    X['day_recorded'] = X['date_recorded'].dt.day
    X = X.drop(columns='date_recorded')
    
    # Engineer feature: how many years from construction_year to date_recorded
    X['years'] = X['year_recorded'] - X['construction_year']
    X['years_MISSING'] = X['years'].isnull()
    
    
    # return the wrangled dataframe
    return X


In [28]:
train = pd.merge(pd.read_csv(DATA_PATH+'waterpumps/train_features.csv'), 
                 pd.read_csv(DATA_PATH+'waterpumps/train_labels.csv'))
test = pd.read_csv(DATA_PATH+'waterpumps/test_features.csv')
sample_submission = pd.read_csv(DATA_PATH+'waterpumps/sample_submission.csv')


# Split train into train & val
train, validate = train_test_split(
    train, 
    train_size=0.80, 
    test_size=0.20, 
    stratify=train['status_group'], 
    random_state=42)

In [29]:
train = wrangle(train)
validate = wrangle(validate)
test = wrangle(test)

In [30]:
train_coordinates = train[train["latitude"].notna()]
validate_coordinates = validate[validate["latitude"].notna()]
test_coordinates = test[test["latitude"].notna()]

In [31]:
train_for_rs = train_coordinates[["id","latitude","longitude"]]
validate_for_rs = validate_coordinates[["id","latitude","longitude"]]
test_for_rs = test_coordinates[["id","latitude","longitude"]]

In [32]:
def save_as_shapefile(dataframe, filename):
    gdf = gpd.GeoDataFrame(dataframe, geometry=gpd.points_from_xy(dataframe.longitude, dataframe.latitude))
    gdf.crs = "EPSG:4326"
    gdf.to_file(filename)

In [33]:
save_as_shapefile(train_for_rs, "/home/alex/data/tanzania-pumps-rasterstats/train.shp")
save_as_shapefile(validate_for_rs, "/home/alex/data/tanzania-pumps-rasterstats/validate.shp")
save_as_shapefile(test_for_rs, "/home/alex/data/tanzania-pumps-rasterstats/test.shp")

In [37]:
boundaries = "/home/alex/data/2012 Wards Shapefiles/TZwards.shp"

raster_folder = "/home/alex/data/CHIRPS-africa-monthly"

identifier_column = "Ward_Name"

file_prefix = "chirps-v2.0."

file_type_of_interest = ".tif"

statistic_of_interest = "mean"

train_output_csv = "/home/alex/data/tanzania-pumps-rasterstats/tanzania_chirps_train.csv"

In [39]:
results = {}

os.chdir(raster_folder)

def processing(name): 
    
    result = zonal_stats(boundaries, name, stats=[statistic_of_interest], nodata=-9999, all_touched=True, geojson_out=False) 
    
    return (name, result)  

with multiprocessing.Pool(multiprocessing.cpu_count()) as pool: 
    
    for name, result in pool.map(processing, glob.glob("*" + file_type_of_interest)): 
        
        results[name] = result